# Donut Fine-tuning

El presente notebook es el proceso de fine-tuning para [DoNut-base](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://huggingface.co/naver-clova-ix/donut-base-finetuned-cord-v2&ved=2ahUKEwjMh4O54vGNAxVsIrkGHQznKskQFnoECBcQAQ&usg=AOvVaw1uKtlO2jgCL6oC_haM4FIB).

In [1]:
pip install datasets transformers

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/juan/anaconda3/envs/CEIA/lib/python3.12/site-packages/pip/__main__.py", line 22, in <module>
    from pip._internal.cli.main import main as _main
  File "/home/juan/anaconda3/envs/CEIA/lib/python3.12/site-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/home/juan/anaconda3/envs/CEIA/lib/python3.12/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/home/juan/anaconda3/envs/CEIA/lib/python3.12/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/home/juan/anaconda3/envs/CEIA/lib/python3.12/site-packages/pip/_internal/build_env.py", line 18, in <module>
    from pip._internal.cli.s

In [2]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
#from transformers import MobileViTForImageClassification, MobileViTImageProcessor
from transformers import ViTForImageClassification, ViTImageProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt
import numpy as np

2025-06-18 19:20:35.321429: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 19:20:35.446327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750285235.498326   66447 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750285235.514007   66447 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750285235.623405   66447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
device =  'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device) # Expected: ‘cuda’ if Linux else ‘mps’ if MacOS
device =  'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

cuda


In [4]:
import transformers

print(torch.__version__)
print(transformers.__version__)

2.6.0+cu126
4.52.4


# Cargando variables de entorno

In [6]:
from dotenv import load_dotenv
import os

# Try different .env paths
env_path = os.path.join("..",".env")
# env path using os path join
    
print(f"🔍 Trying .env path: {env_path}")

# Load environment variables
found = load_dotenv(env_path)
print(f"✅ .env file {'found' if found else 'not found'}")

# Verify loaded variables
print("\n🔎 Environment variables:")
for var in ["PROCESSED_DATA_DIR", "RAW_DATA_DIR", "CHECKPOINT_DIR"]:
    value = os.getenv(var)
    print(f"{var}: {'✅' if value else '❌'} {value}")

img_dir_train = os.path.join(os.getenv("PROCESSED_DATA_DIR"),'train2014')
img_dir_val = os.path.join(os.getenv("PROCESSED_DATA_DIR"),'val2014')
img_dir_test =  os.path.join(os.getenv("PROCESSED_DATA_DIR"),'test2014')
ann_coco_text = os.path.join(os.getenv("RAW_DATA_DIR"),'cocotext.v2.zip')

print(ann_coco_text)

🔍 Trying .env path: ../.env
✅ .env file found

🔎 Environment variables:
PROCESSED_DATA_DIR: ✅ /home/juan/CEIA/vpc3-proyecto/vpc3_proyecto/data/processed
RAW_DATA_DIR: ✅ /home/juan/CEIA/CEIA-ViT/TrabajosPracticos/TP_Final/data/raw
CHECKPOINT_DIR: ✅ /home/juan/CEIA/vpc3-proyecto/vpc3_proyecto/models
/home/juan/CEIA/CEIA-ViT/TrabajosPracticos/TP_Final/data/raw/cocotext.v2.zip


# Evaluamos modelo finetuneado  para ver si mejoro alguna metrica

In [8]:
from vpc3_proyecto.model_training.dataset_donut_model import DonutTextDatasetFromCocoTextV2Raw
from vpc3_proyecto.model_evaluation.utils import get_last_checkpoint_folder
import torch
import glob
from transformers import VisionEncoderDecoderModel, DonutProcessor

torch.cuda.empty_cache()
donut_checkpoint_dir =os.path.join(os.getenv("CHECKPOINT_DIR"),'donut-finetuned-coco2014')#TODO: promptear al usuario detectando las carpetas dentro de checkpint directory para que elija que folder usar / que modelo guardado elegir
last_ckpt = get_last_checkpoint_folder(donut_checkpoint_dir)
manually_saved_folder = os.path.join(last_ckpt, "manually-saved")


processor = DonutProcessor.from_pretrained(manually_saved_folder)

model = VisionEncoderDecoderModel.from_pretrained(
    manually_saved_folder,
    device_map="auto",
    torch_dtype=torch.float16  
)


test_dataset = DonutTextDatasetFromCocoTextV2Raw(
    img_dir_test,  
    ann_coco_text,
    processor=processor,
    max_length=512
)
model.config.decoder_start_token_id = processor.tokenizer.pad_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [27]:
print(f"Model device: {model.device}")
print(f"Vocab size: {model.config}")

Model device: cuda:0
Vocab size: VisionEncoderDecoderConfig {
  "architectures": [
    "VisionEncoderDecoderModel"
  ],
  "decoder": {
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "add_final_layer_norm": true,
    "attention_dropout": 0.0,
    "classifier_dropout": 0.0,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 4,
    "dropout": 0.1,
    "encoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "encoder_layerdrop": 0.0,
    "encoder_layers": 12,
    "init_std": 0.02,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "max_position_embeddings": 768,
    "model_type": "mbart",
    "num_hidden_layers": 12,
    "scale_embedding": true,
    "torch_dtype": "float16",
    "use_cache": true,
    "vocab_size": 57580
  },
  "decoder_start_token_id": 1,
  "encoder": {
    "attention_probs_dropout_prob": 0.0,
    "depths": [
      

In [9]:
from vpc3_proyecto.model_evaluation.utils import manual_evaluate

metrics = manual_evaluate(
    model=model,
    dataset=test_dataset,
    processor=processor,
    max_samples=None  ,
    results_save_path=manually_saved_folder
)

  0%|          | 1/2348 [00:18<12:05:26, 18.55s/it]


KeyboardInterrupt: 

In [ ]:
model